In [ ]:
pip install names

In [ ]:
pip install pandas==1.5.1

In [ ]:
import pandas as pd
import names
import random
import time
from datetime import datetime
import numpy as np
from random import randrange
from datetime import timedelta

In [ ]:
#To generate initial timestamps

def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

Create product table

In [ ]:
# initialize data of lists.
product_types = {'PID': ['P1', 'P2', 'P3'],
        'Value': [15,10,20],
         'Fragile': [0,0,1]}

# Create DataFrame
product_types_df = pd.DataFrame(product_types)

# Print the output.
product_types_df

Create customer table

In [ ]:
customers_df = pd.DataFrame(columns = ["CID", "Name","Bank Account"])
for i in range(50):
    customers_df = customers_df.append({'CID':"c"+str(i+1),
                                        'Name':names.get_full_name(),
                                        'Bank Account': 'BE'+ str(68-i)+" " + str(7689-i*3)+" "  + str(4567+i*13+67)+" "  + str(3467 - i*7 -789)},
                                       ignore_index=True)
customers_df

Create Order table

In [ ]:
orders_df = pd.DataFrame(columns = ["OID","Quantity"])

Create the events table and the dynamic attribute tables

In [ ]:
events_df = pd.DataFrame(columns = ["EID", "Activity","Timestamp",'Customer','Order',"Product Type","Resource"])
quantity_df = pd.DataFrame(columns = ["QID", "P1","P2",'P3',"EID","OID"])
order_value_df = pd.DataFrame(columns = ["VID", "Value","EID","OID"])
refund_df = pd.DataFrame(columns = ["RID", "Refund Value","EID","OID"])
shipping_method_df = pd.DataFrame(columns = ["SID", "Method","EID","OID"])

Create list of potential resources for each activity

In [ ]:
#Resources1 are responsible for the activities: Receive order, confirm purchase, determine shipping method,
#sent invoice, recollect letter, pay refund, file order

Resources1= ['Jan','Gabriel','Olivia','Sophia','Evelyn','Noah']

#Resources 2 are responsible for the activities: Collect product, put protection around product, add product to package

Resources2 = ['Johannes','Elizabeth','Jack','Chloe','Sebastian','Grace']

#Resources 3 are responsible for the activities: Ship by courrier, Ship by mail, ship by express courrier, deliver package

Resources3 = ['Alexandre','Violet','David','Zoe','Ryan','Claire']

In [ ]:
def choice_excluding(lst, exception):
    possible_choices = [v for v in lst if v != exception]
    return random.choice(possible_choices)
print(choice_excluding(Resources1,random.choice(Resources1)))

Create first event

In [ ]:
#Spare  with the idea of having the timestamps overlap
count = 1
Number_of_orders=100
min_value = 100
refund_prob = 0.5
time_between_events = 700
next_event_timerange = 10000

resource1 = random.choice(Resources1)
resource2 = random.choice(Resources2)
resource3 = random.choice(Resources3)

start = datetime.strptime('1/1/2022 08:30 AM', '%d/%m/%Y %I:%M %p')
end =  datetime.strptime('1/1/2022 6:00 PM', '%d/%m/%Y %I:%M %p')

for i in range(0, Number_of_orders):
    #Initialize customers
    customer= customers_df.sample()
    CID=customer['CID'].values
    #Initialize products
    product = product_types_df.sample(n=3,replace=True)
    product_list = set(product.PID.values.tolist())
    print('product_list', product_list)

    #Initialize resources
    resource1 = choice_excluding(Resources1,resource1)
    resource2 = choice_excluding(Resources2,resource2)
    resource3 = choice_excluding(Resources3,resource3)

    print(start,end)

    stamp = random_date(start, end)

# Create first event

    #Events Table
    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Place Order',
                                    'Timestamp': stamp,
                                 'Customer': CID[0],
                                 'Order': 'o'+str(i+1),
                                 'Product Type': product_list},
                                           ignore_index=True)
    # Order Table
    orders_df=orders_df.append({'OID':'o'+str(i+1)},ignore_index=True)
    # Quantity Table
    quantity_df = quantity_df.append({'QID': 'Q' + str(i+1),'EID':"e"+str(i+count),'OID':'o'+str(i+1)},ignore_index=True)
    total = 0
    Quantity = []
    #for q in range(0,3):
    for q in range(0, len(product_types_df.columns)):
            quantity = random.randint(1, 4)
            Quantity.append(quantity)
            value = product.iloc[q]['Value']*quantity
            total = total + value
    product['Quantity'] = Quantity
    q_sum = product.groupby('PID').sum()
    print(q_sum)


    for value in product_list:
        quantity_df.at[i,value] = q_sum.at[value,'Quantity']


    # Refund Table
    refund_df = refund_df.append({'RID':'R'+ str(refund_df.shape[0] + 1), 'Refund Value' : 0,'EID':"e"+str(i+count),'OID':'o'+str(i+1)},ignore_index=True)

    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)



 # Create event 2

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Receive Order',
                                    'Timestamp': stamp,
                                 'Customer': CID[0],
                                 'Order': 'o'+str(i+1),
                                 'Product Type': product_list,
                                 'Resource': resource1},
                                           ignore_index=True)
    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

 # Create event 3

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Confirm Purchase',
                                    'Timestamp': stamp,

                                 'Order': 'o'+str(i+1),
                                 'Product Type': product_list,
                                 'Resource': resource1},
                                           ignore_index=True)

    # Value Table (created by confirm purchase)

    order_value_df = order_value_df.append({'VID':'V'+str(i+1),'Value': total,'EID':"e"+str(i+count),'OID':'o'+str(i+1)},ignore_index=True)

    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

 # Create event 4
    for value in product_list:
        events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Collect product from warehouse',
                                    'Timestamp': stamp,

                                 'Order': 'o'+str(i+1),
                                 'Product Type': set([value]),
                                 'Resource': resource2},
                                           ignore_index=True)
        # Increase count
        count = count + 1

      #Increase timestamp at random between 1 and time_between_events seconds
        time_duration = random.uniform(1,time_between_events)

        stamp = stamp + timedelta(seconds=time_duration)

 # Create event 5
    for value in product_list:
        fragile = product_types_df.loc[product_types_df['PID'] == value]['Fragile'].values
        if fragile[0] == 1:
            events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Put protection around the product',
                                    'Timestamp': stamp,

                                 'Order': 'o'+str(i+1),
                                 'Product Type': set([value]),
                                 'Resource': resource2},
                                           ignore_index=True)
            # Increase count
            count = count + 1

            #Increase timestamp at random between 1 and time_between_events seconds
            time_duration = random.uniform(1,time_between_events)

            stamp = stamp + timedelta(seconds=time_duration)

 # Create event 6
    for value in product_list:
        events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Add product to package',
                                    'Timestamp':  stamp,

                                 'Order': 'o'+str(i+1),
                                 'Product Type': set([value]),
                                 'Resource': resource2},
                                           ignore_index=True)
        # Increase count
        count = count + 1

        #Increase timestamp at random between 1 and time_between_events seconds
        time_duration = random.uniform(1,time_between_events)

        stamp = stamp + timedelta(seconds=time_duration)

# Create event 7
    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Get confirmation',
                                  'Timestamp': stamp,
                                 'Customer': CID[0],
                                 'Order': 'o'+str(i+1)},
                                           ignore_index=True)
    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

# Create event 8
    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Confirm shipping information',
                                  'Timestamp': stamp,
                                 'Customer': CID[0],
                                 'Order': 'o'+str(i+1)},
                                           ignore_index=True)
    # Increase count
    count = count + 1

   #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

# Create event 9
    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Determine shipping method',
                                  'Timestamp': stamp,
                                 'Product Type': product_list,
                                 'Order': 'o'+str(i+1),
                                 'Resource': resource1},
                                           ignore_index=True)
   # Increase count
    count = count + 1

  #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    #make the shipping method table

    #We need to parallelize therefore we make a split with random so either first the courrier is
    #send or first the invoice is send
    split = random.randint(0, 1)

    #Shipping method comes first

    if split==1:
        print('Shipping method comes first')
        #make event Determine shipping method and shipping method event itself
        control=0
        for value in product_list:
            fragile = product_types_df.loc[product_types_df['PID'] == value]['Fragile'].values
            if fragile[0] == 1:
                control = 1
                shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count-1),
                                                        'SID': 'S'+str(shipping_method_df.shape[0] + 1),
                                                        'Method': 'Courrier',
                                                        'OID':'o' + str(i+1)},
                                               ignore_index=True)

                #Add shipping event courrier

                now=datetime.now()
                events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Ship by courrier',
                                      'Timestamp': stamp,

                                     'Order': 'o'+str(i+1),
                                     'Resource': resource3},
                                               ignore_index=True)
                # Increase count
                count = count + 1

                #Increase timestamp at random between 1 and time_between_events seconds
                time_duration = random.uniform(1,time_between_events)

                stamp = stamp + timedelta(seconds=time_duration)

                break

        if order_value_df.loc[i]['Value'] >= min_value and control !=1 :
            control = 1
            shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count-1),
                                                        'SID': 'S' + str(shipping_method_df.shape[0] + 1),
                                                        'Method': 'Courrier',
                                                        'OID':'o' + str(i+1)},
                                                   ignore_index=True)
            #Add shipping event courrier
            now=datetime.now()
            events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Ship by courrier',
                                      'Timestamp': stamp,

                                     'Order': 'o'+str(i+1),
                                                         'Resource': resource3},
                                               ignore_index=True)
            # Increase count
            count = count + 1

            #Increase timestamp at random between 1 and time_between_events seconds
            time_duration = random.uniform(1,time_between_events)

            stamp = stamp + timedelta(seconds=time_duration)

        if control !=1:
            shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count-1),
                                                        'SID': 'S'+str(shipping_method_df.shape[0] + 1),
                                                        'Method': 'Mail',
                                                        'OID':'o' + str(i+1)},
                                                   ignore_index=True)
            #Add shipping event mail
            events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Ship by mail',
                                      'Timestamp': stamp,
                                     'Order': 'o'+str(i+1),
                                                         'Resource': resource3},
                                               ignore_index=True)
            # Increase count
            count = count + 1

            #Increase timestamp at random between 1 and time_between_events seconds
            time_duration = random.uniform(1,time_between_events)

            stamp = stamp + timedelta(seconds=time_duration)

        # Create event send invoice
        events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Send invoice',
                                      'Timestamp': stamp,
                                     'Order': 'o'+str(i+1),
                                     'Resource': resource1},
                                               ignore_index=True)
        # Increase count
        count = count + 1

        #Increase timestamp at random between 1 and time_between_events seconds
        time_duration = random.uniform(1,time_between_events)

        stamp = stamp + timedelta(seconds=time_duration)

    #Sent invoice comes first

    if split==0:
        print('Send invoice comes first')
         # Create event send invoice
        events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Send invoice',
                                      'Timestamp': stamp,
                                     'Order': 'o'+str(i+1),
                                     'Resource': resource1},
                                               ignore_index=True)
        # Increase count
        count = count + 1

        #Increase timestamp at random between 1 and time_between_events seconds
        time_duration = random.uniform(1,time_between_events)

        stamp = stamp + timedelta(seconds=time_duration)

        #make event Determine shipping method and shipping method event itself
        control=0
        for value in product_list:
            fragile = product_types_df.loc[product_types_df['PID'] == value]['Fragile'].values
            if fragile[0] == 1:
                control = 1
                shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count-2),
                                                        'SID': 'S'+str(shipping_method_df.shape[0] + 1),
                                                        'Method': 'Courrier',
                                                        'OID':'o' + str(i+1)},
                                               ignore_index=True)

                #Add shipping event courrier

                now=datetime.now()
                events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Ship by courrier',
                                      'Timestamp': stamp,

                                     'Order': 'o'+str(i+1),
                                     'Resource': resource3},
                                               ignore_index=True)
                # Increase count
                count = count + 1

               #Increase timestamp at random between 1 and time_between_events seconds
                time_duration = random.uniform(1,time_between_events)

                stamp = stamp + timedelta(seconds=time_duration)

                break

        if order_value_df.loc[i]['Value'] >= min_value and control !=1 :
            control = 1
            shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count-2),
                                                        'SID': 'S' + str(shipping_method_df.shape[0] + 1),
                                                        'Method': 'Courrier',
                                                        'OID':'o' + str(i+1)},
                                                   ignore_index=True)
            #Add shipping event courrier
            now=datetime.now()
            events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Ship by courrier',
                                      'Timestamp': stamp,

                                     'Order': 'o'+str(i+1),
                                                         'Resource': resource3},
                                               ignore_index=True)
            # Increase count
            count = count + 1

           #Increase timestamp at random between 1 and time_between_events seconds
            time_duration = random.uniform(1,time_between_events)

            stamp = stamp + timedelta(seconds=time_duration)

        if control !=1:
            shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count-2),
                                                        'SID': 'S'+str(shipping_method_df.shape[0] + 1),
                                                        'Method': 'Mail',
                                                        'OID':'o' + str(i+1)},
                                                   ignore_index=True)
            #Add shipping event mail
            now=datetime.now()
            events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Ship by mail',
                                      'Timestamp': stamp,
                                     'Order': 'o'+str(i+1),
                                                         'Resource': resource3},
                                               ignore_index=True)
            # Increase count
            count = count + 1


            #Increase timestamp at random between 1 and time_between_events seconds
            time_duration = random.uniform(1,time_between_events)

            stamp = stamp + timedelta(seconds=time_duration)

    # Create event deliver package
    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Deliver package',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer':CID[0],
                                                     'Resource': resource3},
                                           ignore_index=True)

    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    # Create event Determine satisfaction
        #randomly decide on whether the customer wants a refund or not
    r = random.uniform(0, 1)
    #r = random.randint(0, 1)

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Determine satisfaction',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer':CID[0]},
                                           ignore_index=True)

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    if r<refund_prob:
        print('Customer does not want refund')
        count = count + 1
        events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'File Order',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Resource': resource1
                                 },
                                           ignore_index=True)


        #Initialize the next timerange
        start = stamp - timedelta(seconds= next_event_timerange)
        end = stamp

        continue

    #if no refund is asked the process ends with file order

    #create event File order
    else:
        refund_df = refund_df.append({'RID':'R' + str(refund_df.shape[0] + 1), 'Refund Value' : 1,'EID':"e"+str(i+count),'OID':'o'+str(i+1)},ignore_index=True)

        print('Since refund is asked, we take the other path')

     # Increase count
    count = count + 1

   #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    # Create event Request refund


    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Request refund',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer':CID[0]},
                                           ignore_index=True)


    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    # Create event Confirm shipping information

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Confirm shipping information',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer':CID[0]},
                                           ignore_index=True)

    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    # Create event Determine shipping method

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Determine shipping method',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Resource': resource1},
                                           ignore_index=True)


    shipping_method_df = shipping_method_df.append({'EID':"e"+str(i+count),
                                                    'SID': 'S'+ str(shipping_method_df.shape[0] + 1),
                                                    'Method': 'Express Courrier',
                                                    'OID':'o' + str(i+1)},
                                               ignore_index=True)

   # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)


    #We need to parallelize therefore we make a split with random so either first the express courrier is
    #send or first the recollect letter is send

    split = random.randint(0, 1)

    if split == 0:
            print('first express courrier then recollect letter')

            # Create event Ship by express courrier

            events_df = events_df.append({'EID':"e"+str(i+count),
                                          'Activity':'Ship by express courrier',
                                          'Timestamp': stamp ,
                                         'Order': 'o'+str(i+1),
                                                             'Resource': resource3},
                                                   ignore_index=True)




            # Increase count
            count = count + 1

            #Increase timestamp at random between 1 and time_between_events seconds
            time_duration = random.uniform(1,time_between_events)

            stamp = stamp + timedelta(seconds=time_duration)

            # Create event Recollect Letter

            events_df = events_df.append({'EID':"e"+str(i+count),
                                          'Activity':'Recollect Letter',
                                          'Timestamp': stamp ,
                                         'Order': 'o'+str(i+1),
                                         'Resource': resource1},
                                                   ignore_index=True)


            # Increase count
            count = count + 1

            #Increase timestamp at random between 1 and time_between_events seconds
            time_duration = random.uniform(1,time_between_events)

            stamp = stamp + timedelta(seconds=time_duration)

    if split == 1:
        print('first recollect letter then express courrier')

         # Create event Recollect Letter

        events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Recollect Letter',
                                      'Timestamp': stamp ,
                                     'Order': 'o'+str(i+1),
                                     'Resource': resource1},
                                               ignore_index=True)


        # Increase count
        count = count + 1

        #Increase timestamp at random between 1 and time_between_events seconds
        time_duration = random.uniform(1,time_between_events)

        stamp = stamp + timedelta(seconds=time_duration)

        # Create event Ship by express courrier

        events_df = events_df.append({'EID':"e"+str(i+count),
                                      'Activity':'Ship by express courrier',
                                      'Timestamp': stamp ,
                                     'Order': 'o'+str(i+1),
                                                         'Resource': resource3},
                                               ignore_index=True)

        # Increase count
        count = count + 1

        #Increase timestamp at random between 1 and time_between_events seconds
        time_duration = random.uniform(1,time_between_events)

        stamp = stamp + timedelta(seconds=time_duration)



    # Create event Return Package

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Return Package',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer': CID[0]},
                                           ignore_index=True)


    # Increase count
    count = count + 1

    ##Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

     # Create event Pay refund

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Pay refund',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer': CID[0],
                                 'Resource': resource1},
                                           ignore_index=True)


    # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    # Create event Deliver package

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Deliver package',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer': CID[0],
                                                     'Resource': resource3},
                                           ignore_index=True)


    # Increase count
    count = count + 1

   #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    #Create event Determine satisfaction

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'Determine satisfaction',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Customer':CID[0]},
                                           ignore_index=True)

    #Change refund variable to 0 since the customer is happy

    refund_df = refund_df.append({'RID':'R' + str(refund_df.shape[0] + 1), 'Refund Value' : 0,'EID':"e"+str(i+count),'OID':'o'+str(i+1)},ignore_index=True)


   # Increase count
    count = count + 1

    #Increase timestamp at random between 1 and time_between_events seconds
    time_duration = random.uniform(1,time_between_events)

    stamp = stamp + timedelta(seconds=time_duration)

    #Create event File Order

    events_df = events_df.append({'EID':"e"+str(i+count),
                                  'Activity':'File Order',
                                  'Timestamp': stamp ,
                                 'Order': 'o'+str(i+1),
                                 'Resource': resource1},
                                           ignore_index=True)


    #Initialize the next timerange
    start = stamp- timedelta(seconds= next_event_timerange)
    end = stamp


Order the events by timestamps

In [ ]:
#First order events by timestamps
events_df = pd.DataFrame(events_df)
events_df = events_df.sort_values(by=['Timestamp'])
#Reset the indexes
events_df= events_df.reset_index(drop=True)
events_df= events_df.reset_index()
#Rename column names
events_df=events_df.rename(columns = {'index':'EID_new', 'EID':'EID_old'})
#Rename the values of 'EID_new'
events_df['EID_new'] ='e' + events_df['EID_new'].astype(str)
events_df = events_df.fillna('/')
events_df.head(50)


In [ ]:
#For every row in the dynamic attribute table
    #take the EID
    #look it up in the events_df table and retrieve the EID_new save it in a separate variable
    # put that value in the attributes table

for j in range(len(quantity_df)):
    ID = quantity_df.loc[j]['EID']
    quantity_df.loc[j]['EID'] = events_df['EID_new'].loc[events_df['EID_old'] == ID].values.tolist()[0]

for j in range(len(order_value_df)):
    ID = order_value_df.loc[j]['EID']
    order_value_df.loc[j]['EID'] = events_df['EID_new'].loc[events_df['EID_old'] == ID].values.tolist()[0]

for j in range(len(refund_df)):
    ID = refund_df.loc[j]['EID']
    refund_df.loc[j]['EID'] = events_df['EID_new'].loc[events_df['EID_old'] == ID].values.tolist()[0]

for j in range(len(shipping_method_df)):
    ID = shipping_method_df.loc[j]['EID']
    shipping_method_df.loc[j]['EID'] = events_df['EID_new'].loc[events_df['EID_old'] == ID].values.tolist()[0]

events_df= events_df.drop(['EID_old'], axis=1)
events_df.rename(columns = {'EID_new':'EID'}, inplace = True)



FIX QUANTITY TABLE


In [ ]:
quantity_df[['P1','P2','P3']] =  quantity_df[['P1','P2','P3']].fillna(0)
quantity_df['Quantity']=quantity_df[['P1','P2','P3']].values.tolist()
quantity_df=quantity_df.drop(['P1','P2','P3'], axis=1)
columns_titles = ["QID","Quantity",'EID','OID']
quantity_df=quantity_df.reindex(columns=columns_titles)
quantity_df

In [ ]:
orders_df["Quantity"]= quantity_df['Quantity']

In [ ]:
orders_df

In [ ]:
product_types_df

In [ ]:
customers_df

In [ ]:
orders_df

In [ ]:
quantity_df

In [ ]:
order_value_df

In [ ]:
refund_df

In [ ]:
shipping_method_df

In [ ]:
events_df

In [ ]:
writer = pd.ExcelWriter('DOCEL log.xlsx', engine='xlsxwriter')

In [ ]:
# event table
events_df.to_excel(writer, sheet_name='Events')

#object tables
customers_df.to_excel(writer, sheet_name='Customer')
product_types_df.to_excel(writer, sheet_name='Product Type')
orders_df.to_excel(writer, sheet_name='Order')

#Attribute tables
#quantity_df.to_excel(writer, sheet_name='Quantity')
order_value_df.to_excel(writer, sheet_name='Value')
refund_df.to_excel(writer, sheet_name='Refund Value')
shipping_method_df.to_excel(writer, sheet_name='Method')

In [ ]:
# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
#OCEL LOG


In [ ]:
#OCEL = pd.merge(, quantity_df, how="left",on='EID')
OCEL = pd.merge(events_df, order_value_df, how="left",on='EID')
OCEL = pd.merge(OCEL, refund_df, how="left",on='EID')
OCEL = pd.merge(OCEL, shipping_method_df, how="left",on='EID')
OCEL

In [ ]:

OCEL=OCEL.fillna('/')
OCEL=OCEL.drop(['OID_x','OID_y','VID','RID','SID'], axis=1)
OCEL

In [ ]:
writer = pd.ExcelWriter('OCEL log.xlsx', engine='xlsxwriter')

In [ ]:
# event table
OCEL.to_excel(writer, sheet_name='Events')

#object tables
customers_df.to_excel(writer, sheet_name='Customer')
product_types_df.to_excel(writer, sheet_name='Product Type')
orders_df.to_excel(writer, sheet_name='Order')

In [ ]:
# Close the Pandas Excel writer and output the Excel file.
writer.save()